
<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Random Forests and ExtraTrees


_Authors: Matt Brems (DC), Riley Dallas (AUS)_

---

## Random Forests
---

With bagged decision trees, we generate many different trees on pretty similar data. These trees are **strongly correlated** with one another. Because these trees are correlated with one another, they will have high variance. Looking at the variance of the average of two random variables $T_1$ and $T_2$:

$$
\begin{eqnarray*}
Var\left(\frac{T_1+T_2}{2}\right) &=& \frac{1}{4}\left[Var(T_1) + Var(T_2) + 2Cov(T_1,T_2)\right]
\end{eqnarray*}
$$

If $T_1$ and $T_2$ are highly correlated, then the variance will about as high as we'd see with individual decision trees. By "de-correlating" our trees from one another, we can drastically reduce the variance of our model.

That's the difference between bagged decision trees and random forests! We're going to do the same thing as before, but we're going to de-correlate our trees. This will reduce our variance (at the expense of a small increase in bias) and thus should greatly improve the overall performance of the final model.

So how do we "de-correlate" our trees?

Random forests differ from bagging decision trees in only one way: they use a modified tree learning algorithm that selects, at each split in the learning process, a **random subset of the features**. This process is sometimes called the *random subspace method*.

The reason for doing this is the correlation of the trees in an ordinary bootstrap sample: if one or a few features are very strong predictors for the response variable (target output), these features will be used in many/all of the bagged decision trees, causing them to become correlated. By selecting a random subset of features at each split, we counter this correlation between base trees, strengthening the overall model.

For a problem with $p$ features, it is typical to use:

- $\sqrt{p}$ (rounded down) features in each split for a classification problem.
- $p/3$ (rounded down) with a minimum node size of 5 as the default for a regression problem.

While this is a guideline, Hastie and Tibshirani (authors of Introduction to Statistical Learning and Elements of Statistical Learning) have suggested this as a good rule in the absence of some rationale to do something different.

Random forests, a step beyond bagged decision trees, are **very widely used** classifiers and regressors. They are relatively simple to use because they require very few parameters to set and they perform pretty well.
- It is quite common for interviewers to ask how a random forest is constructed or how it is superior to a single decision tree.

--- 

## Extremely Randomized Trees (ExtraTrees)
Adding another step of randomization (and thus de-correlation) yields extremely randomized trees, or _ExtraTrees_. Like Random Forests, these are trained using the random subspace method (sampling of features). However, they are trained on the entire dataset instead of bootstrapped samples. A layer of randomness is introduced in the way the nodes are split. Instead of computing the locally optimal feature/split combination (based on, e.g., information gain or the Gini impurity) for each feature under consideration, a random value is selected for the split. This value is selected from the feature's empirical range.

This further reduces the variance, but causes an increase in bias. If you're considering using ExtraTrees, you might consider this to be a hyperparameter you can tune. Build an ExtraTrees model and a Random Forest model, then compare their performance!

That's exactly what we'll do below.

## Import libraries
---

We'll need the following libraries for today's lecture:
- `pandas`
- `numpy`
- `GridSearchCV`, `train_test_split` and `cross_val_score` from `sklearn`'s `model_selection` module 
- `RandomForestClassifier` and `ExtraTreesClassifier` from `sklearn`'s `ensemble` module 

In [ ]:
import pandas as pd
import numpy as np
from sklearn import set_config
import matplotlib.pyplot as plt
set_config(display='diagram') 
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

## Load Data
---

Our first dataset has to do with detecting sarcasm in news headlines. [Source](https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection)

In [ ]:
# import json

# def parse_data(file):
#     for l in open(file,'r'):
#         yield json.loads(l)

# data = list(parse_data('data/Sarcasm_Headlines_Dataset.json'))

In [ ]:
# data[:2]

In [ ]:
#turn our list of data into a DataFrame...
# sarcasm_df = pd.DataFrame(data)

In [8]:
# load csv 
sarcasm_df = pd.read_csv('data/sarcasm.csv')
sarcasm_df.head()

,link,headline,sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [9]:
print(sarcasm_df.iloc[3, 1])

boehner just wants wife to listen, not come up with alternative debt-reduction ideas


### this is a classification problem- can we predict if a headline is sarcastic?
### supervised- we have a label

## Challenge: What is our baseline accuracy?
---

The baseline accuracy is the percentage of the majority class, regardless of whether it is 1 or 0. It serves as the benchmark for our model to beat.

In [11]:
sarcasm_df['sarcastic'].mean()

0.43895316185555433

In [12]:
sarcasm_df['sarcastic'].mode()

0    0
dtype: int64

In [14]:
sarcasm_df['sarcastic'].value_counts(normalize=True)
# if we guessed sarcastic every time, we would be correct 56% of the time

0    0.561047
1    0.438953
Name: sarcastic, dtype: float64

In [ ]:
baseline_test_predictions = np.ones(y_test.shape) * np.mean(y_train)

In [15]:
X= sarcasm_df[['headline']]

In [16]:
y=sarcasm_df['sarcastic']

## Train/Test Split
---



In [40]:
X_train, X_test, y_train, y_test = train_test_split(sarcasm_df['headline'], sarcasm_df['sarcastic'], stratify=sarcasm_df['sarcastic'], random_state= 42)

## Model instantiation
---

Create an instance of `RandomForestClassifier` and `ExtraTreesClassifier`.

In [41]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [42]:
cvect = CountVectorizer(max_features=1000)

In [43]:
# fit and transform train
X_train_vect = cvect.fit_transform(X_train)
# only transform test
X_test_vect = cvect.transform(X_test)

In [44]:
# n_forest is the number of decision trees. we can grid search this parameter. 
forest = RandomForestClassifier()
extra = ExtraTreesClassifier()

In [45]:
forest.fit(X_train_vect, y_train)

RandomForestClassifier()

In [46]:
extra.fit(X_train_vect, y_train)

ExtraTreesClassifier()

## Model Evaluation
---

Which one has a higher `cross_val_score`?

In [47]:
cross_val_score(forest, X_train_vect, y_train)
# this is a classification problem, these are accuracy scores
# so we're doing better than the baseline- we're predicting correct about 78% of the time 
# these scores also look really steady

array([0.78787123, 0.78407389, 0.78282576, 0.78881677, 0.78082876])

In [49]:
cross_val_score(extra, X_train_vect, y_train)


array([0.79261293, 0.78507239, 0.78507239, 0.78132801, 0.78432351])

## Grid Search
---



In [ ]:
#n_estimators = [10, 100, 500, 1000]
#max_depth = [1, 2, 3, 4]

In [55]:
# set the params we want to test. 
params = {
    'n_estimators': [10, 100, 500, 1000],
    'max_depth': [10, 20]
}



In [56]:
# create the grid
grid = GridSearchCV(forest, param_grid=params)

In [57]:
# this tried all of the params and identified the best combo
# so when we score, we will be scoringo n the best combo of all of these parama
grid.fit(X_train_vect, y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [10, 20],
                         'n_estimators': [10, 100, 500, 1000]})

In [ ]:
grid.score(X_test_vect, y_test)

In [58]:
# n_estimator is the numbr of trees in the forest. 
grid.best_estimator_

RandomForestClassifier(max_depth=20, n_estimators=500)

### Problem

In small groups you are to build a simple classifier to predict customer churn.  Don't get hung up in too much preprocessing; your main goal is to compare a `KNeighborsClassifier`, `LogisticRegressor`, `DecisionTreeClassifier`, and `RandomForestClassifier`.  


If you have time, investigate the feature importances of the decision tree and random forest.  What features do they suggest are the most important to predicting customer churn?

In [59]:
churn_df = pd.read_csv('data/cell_phone_churn.csv')
churn_df.head(2)

,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,eve_mins,eve_calls,eve_charge,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,churn
0,KS,128,415,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.7,1,False
1,OH,107,415,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.7,1,False


In [77]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [71]:
churn_df['churn'].value_counts(normalize=True)

False    0.855086
True     0.144914
Name: churn, dtype: float64

In [80]:
X = churn_df.drop(['vmail_plan', 'intl_plan', 'state', 'churn'], axis = 1)
y = churn_df['churn']

In [81]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=churn_df['churn'], random_state=42)

In [82]:
# scale the X data
scaler = StandardScaler()

# fit and transform the train data
X_train_scaled = scaler.fit_transform(X_train)

# only transform the test data
X_test_scaled = scaler.transform(X_test)

### KNN

In [90]:
# instantiate
knn = KNeighborsClassifier()

# fit
knn.fit(X_train_scaled, y_train)

# predict
knn_preds = knn.predict(X_test_scaled)

# compare the predictions on the X_test to the actuals within the y_test
# mse is not a metric for classification, instead we look at accuracy here using the score 
knn.score(X_test_scaled, y_test)

0.8836930455635491

### Logistic Regressor 

In [94]:
# instantiate
lgr = LogisticRegression()

# fit
lgr.fit(X_train_scaled, y_train)

# predict 
lrg_preds= lgr.predict(X_test_scaled)

# compre the predictions 
lgr.score(X_test_scaled, y_test)


0.854916067146283

### Decision Tree Classifier

In [95]:
# instantiate
dtr = DecisionTreeClassifier()

# fit 
dtr.fit(X_train_scaled, y_train)

# predict
dtr_preds= dtr.predict(X_test_scaled)

# compare the predictions
dtr.score(X_test_scaled, y_test)


0.8693045563549161

### Random Forest Classifier

In [96]:
# instantiate
rfr = RandomForestClassifier()

# fit 
rfr.fit(X_train_scaled, y_train)

# predict
rfr_preds= rfr.predict(X_test_scaled)

# compare the predictions
rfr.score(X_test_scaled, y_test)

0.919664268585132


### Summary
- Random forest classifier seems to perform the best in this situation

In [97]:
from sklearn.ensemble import VotingClassifier

In [ ]:
voter = VotingClassifier([('knn', knn_pipe)])